# Load 'train.csv'

train.csv 的資料為 12 個月中，每個月取 20 天，每天 24 小時的資料(每小時資料有 18 個 features)。

In [ ]:
# Load 'train.csv'
import sys
import pandas as pd
import numpy as np
from google.colab import drive 
!gdown --id '1wNKAxQ29G15kgpBy_asjTcZRRgmsCZRm' --output data.zip
!unzip data.zip
# data = pd.read_csv('gdrive/My Drive/hw1-regression/train.csv', header = None, encoding = 'big5')
data = pd.read_csv('./train.csv', encoding = 'big5')

print("loaded")

Downloading...
From: https://drive.google.com/uc?id=1wNKAxQ29G15kgpBy_asjTcZRRgmsCZRm
To: /content/data.zip
100% 177k/177k [00:00<00:00, 54.7MB/s]
Archive:  data.zip
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: test.csv                
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train.csv               
loaded


# Preprocessing

取需要的數值部分，將 'RAINFALL' 欄位全部補 0。 另外，如果要在 colab 重覆這段程式碼的執行，請從頭開始執行(把上面的都重新跑一次)，以避免跑出不是自己要的結果（若自己寫程式不會遇到，但 colab 重複跑這段會一直往下取資料。意即第一次取原本資料的第三欄之後的資料，第二次取第一次取的資料掉三欄之後的資料，...）。

In [ ]:
# Preprocessing
data = data.iloc[:, 3:]
data[data == 'NR'] = 0
raw_data = data.to_numpy()
print(raw_data)

print("Preprocessed")

[['14' '14' '14' ... '15' '15' '15']
 ['1.8' '1.8' '1.8' ... '1.8' '1.8' '1.8']
 ['0.51' '0.41' '0.39' ... '0.35' '0.36' '0.32']
 ...
 ['36' '55' '72' ... '118' '100' '105']
 ['1.9' '2.4' '1.9' ... '1.5' '2' '2']
 ['0.7' '0.8' '1.8' ... '1.6' '1.8' '2']]
Preprocessed


# Extract Features (1)

將原始 4320 * 18 的資料依照每個月分重組成 12 個 18 (features) * 480 (hours) 的資料。

In [ ]:
# Extract Features (1)
month_data = {}
for month in range(12):
    sample = np.empty([18, 480])
    for day in range(20):
        sample[:, day * 24 : (day + 1) * 24] = raw_data[18 * (20 * month + day) : 18 * (20 * month + day + 1), :]
    month_data[month] = sample
    
print(month_data)
print("Extracted Features (1)")

{0: array([[14.  , 14.  , 14.  , ..., 14.  , 13.  , 13.  ],
       [ 1.8 ,  1.8 ,  1.8 , ...,  1.8 ,  1.8 ,  1.8 ],
       [ 0.51,  0.41,  0.39, ...,  0.34,  0.41,  0.43],
       ...,
       [35.  , 79.  ,  2.4 , ..., 48.  , 63.  , 53.  ],
       [ 1.4 ,  1.8 ,  1.  , ...,  1.1 ,  1.9 ,  1.9 ],
       [ 0.5 ,  0.9 ,  0.6 , ...,  1.2 ,  1.2 ,  1.3 ]]), 1: array([[ 15.  ,  14.  ,  14.  , ...,   8.4 ,   8.  ,   7.6 ],
       [  1.8 ,   1.8 ,   1.7 , ...,   1.7 ,   1.7 ,   1.7 ],
       [  0.27,   0.26,   0.25, ...,   0.36,   0.35,   0.32],
       ...,
       [113.  , 109.  , 104.  , ...,  72.  ,  65.  ,  69.  ],
       [  2.3 ,   2.2 ,   2.6 , ...,   1.9 ,   2.9 ,   1.5 ],
       [  2.5 ,   2.2 ,   2.2 , ...,   0.9 ,   1.6 ,   1.1 ]]), 2: array([[ 18.  ,  18.  ,  18.  , ...,  14.  ,  13.  ,  13.  ],
       [  1.8 ,   1.8 ,   1.8 , ...,   1.8 ,   1.8 ,   1.8 ],
       [  0.39,   0.36,   0.4 , ...,   0.42,   0.47,   0.49],
       ...,
       [103.  , 128.  , 115.  , ...,  60.  ,  94.  ,  53

# Extract Features (2)

每個月會有 480hrs，每 9 小時形成一個 data，每個月會有 471 個 data，故總資料數為 471 * 12 筆，而每筆 data 有 9 * 18 的 features (一小時 18 個 features * 9 小時)。
對應的 target 則有 471 * 12 個(第 10 個小時的 PM2.5)

In [ ]:
# Extract Features (2)
x = np.empty([12 * 471, 18 * 9], dtype = float)
y = np.empty([12 * 471, 1], dtype = float)
for month in range(12):
    for day in range(20):
        for hour in range(24):
            if day == 19 and hour > 14:
                continue
            x[month * 471 + day * 24 + hour, :] = month_data[month][:,day * 24 + hour : day * 24 + hour + 9].reshape(1, -1) #vector dim:18*9 (9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9)
            y[month * 471 + day * 24 + hour, 0] = month_data[month][9, day * 24 + hour + 9] #value
print(x)
print(y)

print("Extract Features (2)")

[[14.  14.  14.  ...  2.   2.   0.5]
 [14.  14.  13.  ...  2.   0.5  0.3]
 [14.  13.  12.  ...  0.5  0.3  0.8]
 ...
 [17.  18.  19.  ...  1.1  1.4  1.3]
 [18.  19.  18.  ...  1.4  1.3  1.6]
 [19.  18.  17.  ...  1.3  1.6  1.8]]
[[30.]
 [41.]
 [44.]
 ...
 [17.]
 [24.]
 [29.]]
Extract Features (2)


# Normalize (1)

In [ ]:
# Normalize (1)
mean_x = np.mean(x, axis = 0) #18 * 9 
std_x = np.std(x, axis = 0) #18 * 9 
for i in range(len(x)): #12 * 471
    for j in range(len(x[0])): #18 * 9 
        if std_x[j] != 0:
            x[i][j] = (x[i][j] - mean_x[j]) / std_x[j]
print(x)

print("Normalized (1)")

[[-1.35825331 -1.35883937 -1.359222   ...  0.26650729  0.2656797
  -1.14082131]
 [-1.35825331 -1.35883937 -1.51819928 ...  0.26650729 -1.13963133
  -1.32832904]
 [-1.35825331 -1.51789368 -1.67717656 ... -1.13923451 -1.32700613
  -0.85955971]
 ...
 [-0.88092053 -0.72262212 -0.56433559 ... -0.57693779 -0.29644471
  -0.39079039]
 [-0.7218096  -0.56356781 -0.72331287 ... -0.29578943 -0.39013211
  -0.1095288 ]
 [-0.56269867 -0.72262212 -0.88229015 ... -0.38950555 -0.10906991
   0.07797893]]
Normalized (1)


# Split Training Data Into "train_set" and "validation_set"

這部分是針對作業中 report 的第二題、第三題做的簡單示範，以生成比較中用來訓練的 train_set 和不會被放入訓練、只是用來驗證的 validation_set。

In [ ]:
# Split Training Data Into "train_set" and "validation_set"
import math
x_train_set = x[: math.floor(len(x) * 0.8), :]
y_train_set = y[: math.floor(len(y) * 0.8), :]
x_validation = x[math.floor(len(x) * 0.8): , :]
y_validation = y[math.floor(len(y) * 0.8): , :]
print(x_train_set)
print(y_train_set)
print(x_validation)
print(y_validation)
print(len(x_train_set))
print(len(y_train_set))
print(len(x_validation))
print(len(y_validation))

print("Splited")

[[-1.35825331 -1.35883937 -1.359222   ...  0.26650729  0.2656797
  -1.14082131]
 [-1.35825331 -1.35883937 -1.51819928 ...  0.26650729 -1.13963133
  -1.32832904]
 [-1.35825331 -1.51789368 -1.67717656 ... -1.13923451 -1.32700613
  -0.85955971]
 ...
 [ 0.86929969  0.70886668  0.38952809 ...  1.39110073  0.2656797
  -0.39079039]
 [ 0.71018876  0.39075806  0.07157353 ...  0.26650729 -0.39013211
  -0.39079039]
 [ 0.3919669   0.07264944  0.07157353 ... -0.38950555 -0.39013211
  -0.85955971]]
[[30.]
 [41.]
 [44.]
 ...
 [ 7.]
 [ 5.]
 [14.]]
[[ 0.07374504  0.07264944  0.07157353 ... -0.38950555 -0.85856912
  -0.57829812]
 [ 0.07374504  0.07264944  0.23055081 ... -0.85808615 -0.57750692
   0.54674825]
 [ 0.07374504  0.23170375  0.23055081 ... -0.57693779  0.54674191
  -0.1095288 ]
 ...
 [-0.88092053 -0.72262212 -0.56433559 ... -0.57693779 -0.29644471
  -0.39079039]
 [-0.7218096  -0.56356781 -0.72331287 ... -0.29578943 -0.39013211
  -0.1095288 ]
 [-0.56269867 -0.72262212 -0.88229015 ... -0.3895055

# Training

因為常數項的存在，所以 dimension (dim) 需要多加一欄；eps 項是避免 adagrad 的分母為 0 而加的極小數值。
每一個 dimension (dim) 會對應到各自的 gradient, weight (w)，透過一次次的 iteration (iter_time) 學習。

In [ ]:
# Training
dim = 18 * 9 + 1
w = np.zeros([dim, 1])
x = np.concatenate((np.ones([12 * 471, 1]), x), axis = 1).astype(float)
learning_rate = 100
iter_time = 1000
adagrad = np.zeros([dim, 1])
eps = 0.0000000001
for t in range(iter_time):
    loss = np.sqrt(np.sum(np.power(np.dot(x, w) - y, 2))/471/12)#rmse
    if(t%100==0):
        print(str(t) + ":" + str(loss))
    gradient = 2 * np.dot(x.transpose(), np.dot(x, w) - y) #dim*1
    adagrad += gradient ** 2
    w = w - learning_rate * gradient / np.sqrt(adagrad + eps)
np.save('weight.npy', w)
print(w)

print("Training")

0:27.071214829194115
100:33.78905859777454
200:19.91375129819709
300:13.53106819368969
400:10.64546615844617
500:9.277353455475065
600:8.5180420459565
700:8.014061987588422
800:7.63675682477569
900:7.3365637403711235
[[ 2.13740269e+01]
 [ 3.58888909e+00]
 [ 4.56386323e+00]
 [ 2.16307023e+00]
 [-6.58545223e+00]
 [-3.38885580e+01]
 [ 3.22235518e+01]
 [ 3.49340354e+00]
 [-4.60308671e+00]
 [-1.02374754e+00]
 [-3.96791501e-01]
 [-1.06908800e-01]
 [ 2.22488184e-01]
 [ 8.99634117e-02]
 [ 1.31243105e-01]
 [ 2.15894989e-02]
 [-1.52867263e-01]
 [ 4.54087776e-02]
 [ 5.20999235e-01]
 [ 1.60824213e-01]
 [-3.17709451e-02]
 [ 1.28529025e-02]
 [-1.76839437e-01]
 [ 1.71241371e-01]
 [-1.31190032e-01]
 [-3.51614451e-02]
 [ 1.00826192e-01]
 [ 3.45018257e-01]
 [ 4.00130315e-02]
 [ 2.54331382e-02]
 [-5.04425219e-01]
 [ 3.71483018e-01]
 [ 8.46357671e-01]
 [-8.11920428e-01]
 [-8.00217575e-02]
 [ 1.52737711e-01]
 [ 2.64915130e-01]
 [-5.19860416e-02]
 [-2.51988315e-01]
 [ 3.85246517e-01]
 [ 1.65431451e-01]
 [-7

In [ ]:
# Testing

# testdata = pd.read_csv('gdrive/My Drive/hw1-regression/test.csv', header = None, encoding = 'big5')
testdata = pd.read_csv('./test.csv', header = None, encoding = 'big5')
test_data = testdata.iloc[:, 2:]
test_data[test_data == 'NR'] = 0
test_data = test_data.to_numpy()
test_x = np.empty([240, 18*9], dtype = float)
for i in range(240):
    test_x[i, :] = test_data[18 * i: 18* (i + 1), :].reshape(1, -1)
for i in range(len(test_x)):
    for j in range(len(test_x[0])):
        if std_x[j] != 0:
            test_x[i][j] = (test_x[i][j] - mean_x[j]) / std_x[j]
test_x = np.concatenate((np.ones([240, 1]), test_x), axis = 1).astype(float)
test_x

print("tested")

tested


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:2986: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(-key, value, inplace=True)


In [ ]:
# Prediction

w = np.load('weight.npy')
ans_y = np.dot(test_x, w)
ans_y

print("prediction")

prediction


In [ ]:
# Save Prediction to CSV File
import csv
with open('submit.csv', mode='w', newline='') as submit_file:
    csv_writer = csv.writer(submit_file)
    header = ['id', 'value']
    print(header)
    csv_writer.writerow(header)
    for i in range(240):
        row = ['id_' + str(i), ans_y[i][0]]
        csv_writer.writerow(row)
        print(row)

print("Saved prediction")

['id', 'value']
['id_0', 53.25836874148928]
['id_1', 267.6908341577875]
['id_2', 415.81195223214377]
['id_3', 149.19958910274923]
['id_4', 363.9741647096341]
['id_5', 339.45279014058025]
['id_6', 398.21070011279295]
['id_7', 552.0090030987983]
['id_8', 265.681530903415]
['id_9', 1000.3245975181026]
['id_10', 215.1791114129934]
['id_11', 215.00899987389275]
['id_12', 1148.0083952694688]
['id_13', 976.7468763586982]
['id_14', 431.78475659745334]
['id_15', 192.11088212003298]
['id_16', 515.5803577489429]
['id_17', 1176.6430417376273]
['id_18', -7.8887862889789915]
['id_19', 292.4171432107855]
['id_20', 674.2978728506574]
['id_21', 1169.6367230378016]
['id_22', 123.01227123585498]
['id_23', 291.4703498820058]
['id_24', 227.8932165021574]
['id_25', 741.9393653228557]
['id_26', 293.75219882480695]
['id_27', 1338.9823912064128]
['id_28', 166.94872414460855]
['id_29', 950.5080930661551]
['id_30', 353.89125337266125]
['id_31', 153.55220798911074]
['id_32', -1.053457565535031]
['id_33', 400.4909